In [1]:
import os
# os.environ["KERAS_BACKEND"] = "torch"
import keras
from keras import layers



daten von disk laden

In [2]:
train_ds, val_ds = keras.utils.image_dataset_from_directory(
        os.path.join("data", "images"),
        labels="inferred",
        label_mode="categorical",
        color_mode="rgb",
        batch_size=64,
        image_size=(128, 128),
        shuffle=True,
        interpolation="bilinear",
        crop_to_aspect_ratio=True,
        validation_split=0.2,
        subset="both",
        seed=42,
    )
train_ds, val_ds

Found 9100 files belonging to 7 classes.
Using 7280 files for training.
Using 1820 files for validation.


(<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 7), dtype=tf.float32, name=None))>,
 <_PrefetchDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.uint8, name=None), TensorSpec(shape=(None, 7), dtype=tf.float32, name=None))>)

data augmentation

In [7]:
data_augmentation_layers = [
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.Rescaling(1./255)
]

def data_augmentation(images):
    for layer in data_augmentation_layers:
        images = layer(images)
    return images

augmented_train_ds = train_ds.map(
    lambda x, y: (data_augmentation(x), y))

augmented_val_ds = val_ds.map(
    lambda x, y: (data_augmentation(x), y))

augmented_train_ds, augmented_val_ds

(<_MapDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7), dtype=tf.float32, name=None))>,
 <_MapDataset element_spec=(TensorSpec(shape=(None, 128, 128, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 7), dtype=tf.float32, name=None))>)

In [8]:
model = keras.Sequential([
    layers.InputLayer(shape=(128, 128, 3)),
    layers.Conv2D(32, (4, 4), activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(64, (4, 4), activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(128, (4, 4), activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Conv2D(256, (4, 4), activation="relu"),
    layers.MaxPooling2D(pool_size=(2,2)),
    layers.Flatten(),
    layers.Dense(7, activation='softmax'),
])
model.compile(
  'adam',
  loss='categorical_crossentropy',
  metrics=['accuracy'],
)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 125, 125, 32)   │         1,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 62, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 59, 59, 64)     │        32,832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 29, 29, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 26, 26, 128)    │       131,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 13, 13, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_3 (Conv2D)               │ (None, 10, 10, 256)    │       524,544 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 5, 5, 256)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 6400)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 7)              │        44,807 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 734,951 (2.80 MB)

 Trainable params: 734,951 (2.80 MB)

 Non-trainable params: 0 (0.00 B)

In [9]:
from livelossplot import PlotLossesKeras

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5,
    callbacks=[PlotLossesKeras()],
)

accuracy
	training         	 (min:    0.403, max:    0.403, cur:    0.403)
	validation       	 (min:    0.525, max:    0.525, cur:    0.525)
Loss
	training         	 (min:    3.847, max:    3.847, cur:    3.847)
	validation       	 (min:    1.301, max:    1.301, cur:    1.301)
114/114 ━━━━━━━━━━━━━━━━━━━━ 27s 228ms/step - accuracy: 0.2899 - loss: 10.8875 - val_accuracy: 0.5253 - val_loss: 1.3015
Epoch 2/5
 32/114 ━━━━━━━━━━━━━━━━━━━━ 18s 221ms/step - accuracy: 0.5223 - loss: 1.2805

KeyboardInterrupt: 